In [3]:
import os

In [4]:
%pwd

'/Users/somesh/Downloads/code/end_to_end_project/research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'/Users/somesh/Downloads/code/end_to_end_project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass 
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path





In [14]:
from src.data_science_project.constants import *
from src.data_science_project.utils.common import read_yaml, create_directories

In [16]:

class ConfigurationManager: 
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
    
    

In [18]:
import requests 
import logging 
import os 
from pathlib import Path 
from zipfile import ZipFile 


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self): 
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_URL)
            filename = os.path.basename(self.config.local_data_file)
            path_to_download = os.path.join(self.config.root_dir, filename)
            with open(path_to_download, 'wb') as f:
                f.write(response.content)
        else:
            logging.info(f"File already exists of size: {Path(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(file=self.config.local_data_file, mode="r") as zip_ref:
            zip_ref.extractall(path=unzip_path)





In [21]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-13 16:22:47,515: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-08-13 16:22:47,518: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-13 16:22:47,519: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-13 16:22:47,520: INFO: common: created directory at: artifacts/data_ingestion]
